In [1]:
from pathlib import Path

import torch
import pytorch_lightning as pl
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split

In [2]:
class ApartmentDataset(Dataset):
    def __init__(self, features, labels):
        assert len(features) == len(labels)
        self._features = torch.from_numpy(features).float()
        self._labels = torch.from_numpy(labels).float()

    def __len__(self):
        return len(self._features)

    def __getitem__(self, idx):
        return self._features[idx], self._labels[idx]
    

class ApartmentDataModule(pl.LightningDataModule):
    def __init__(self, dataset: ApartmentDataset):
        super().__init__()
        self._dataset = dataset
        self._train_dataset = None
        self._val_dataset = None
        
    def prepare_data(self):
        train_size = round(len(self._dataset) * 0.85)
        self._train_dataset, self._val_dataset = random_split(self._dataset, (train_size, len(self._dataset) - train_size))
    
    def train_dataloader(self):
        return DataLoader(self._train_dataset, batch_size=512, shuffle=True, num_workers=4)

    def val_dataloader(self):
        return DataLoader(self._val_dataset, batch_size=32, num_workers=4)
        

In [3]:
class ApartmentClassifier(pl.LightningModule):
    def __init__(
            self,
            input_size,
            output_size,
    ):
        super(ApartmentClassifier, self).__init__()

        self.l1 = nn.Linear(input_size, 128)
        self.l2 = nn.Linear(128, 64)
        self.l3 = nn.Linear(64, 32)
        self.l4 = nn.Linear(32, output_size)
        
        self.loss = torch.nn.CrossEntropyLoss(weight=torch.FloatTensor([0.45, 0.1, 0.45]).cuda())

    def forward(self, x: torch.Tensor):
        # todo batchnorm

        x = self.l1(x)
        x = torch.relu(x)
        x = self.l2(x)
        x = torch.relu(x)
        x = self.l3(x)
        x = torch.relu(x)
        x = self.l4(x)
        probas = torch.nn.functional.softmax(x, dim=1)
        return probas.float()
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3, weight_decay=1e-6)
    
    def cross_entropy_loss(self, preds, labels):
        return self.loss(preds, labels)
    
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        preds = self.forward(x)
        loss = self.cross_entropy_loss(preds, y)
        # this goes straight to TensorBoard
        self.log('train_loss', loss, on_step=False, on_epoch=True)
        return loss
    
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        preds = self.forward(x)
        loss = self.cross_entropy_loss(preds, y)
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        return loss

In [4]:
import pandas as pd

In [5]:
TRAIN_DATA = Path('data/train_data.csv')

_df = pd.read_csv(TRAIN_DATA)
_df

,SalePrice,YearBuilt,Size(sqf),Floor,HallwayType,HeatingType,AptManageType,N_Parkinglot(Ground),N_Parkinglot(Basement),TimeToBusStop,TimeToSubway,N_manager,N_elevators,SubwayStation,N_FacilitiesInApt,N_FacilitiesNearBy(Total),N_SchoolNearBy(Total)
0,141592,2006,814,3,terraced,individual_heating,management_in_trust,111.0,184.0,5min~10min,10min~15min,3.0,0.0,Kyungbuk_uni_hospital,5,6.0,9.0
1,51327,1985,587,8,corridor,individual_heating,self_management,80.0,76.0,0~5min,5min~10min,2.0,2.0,Daegu,3,12.0,4.0
2,48672,1985,587,6,corridor,individual_heating,self_management,80.0,76.0,0~5min,5min~10min,2.0,2.0,Daegu,3,12.0,4.0
3,380530,2006,2056,8,terraced,individual_heating,management_in_trust,249.0,536.0,0~5min,0-5min,5.0,11.0,Sin-nam,5,3.0,7.0
4,78318,1992,644,2,mixed,individual_heating,self_management,142.0,79.0,5min~10min,15min~20min,4.0,8.0,Myung-duk,3,9.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4119,570796,2007,1928,24,terraced,individual_heating,management_in_trust,0.0,1270.0,0~5min,0-5min,14.0,16.0,Kyungbuk_uni_hospital,10,9.0,10.0
4120,307079,2015,644,22,terraced,individual_heating,management_in_trust,102.0,400.0,0~5min,5min~10min,5.0,10.0,Daegu,7,7.0,11.0
4121,357522,2007,868,20,terraced,individual_heating,management_in_trust,0.0,1270.0,0~5min,0-5min,14.0,16.0,Kyungbuk_uni_hospital,10,9.0,10.0
4122,312389,1978,1327,1,corridor,individual_heating,self_management,87.0,0.0,0~5min,0-5min,1.0,4.0,Kyungbuk_uni_hospital,3,7.0,11.0


In [6]:
categorical_cols = [
    'HallwayType',
    'HeatingType',
    'AptManageType',
    'N_Parkinglot(Ground)',
    'N_Parkinglot(Basement)',
    'TimeToBusStop',
    'TimeToSubway',
    'SubwayStation',
]

df = pd.concat([_df.drop(columns=categorical_cols), *[pd.get_dummies(_df[col], prefix=col, dtype=int) for col in categorical_cols]], axis=1)
df

,SalePrice,YearBuilt,Size(sqf),Floor,N_manager,N_elevators,N_FacilitiesInApt,N_FacilitiesNearBy(Total),N_SchoolNearBy(Total),HallwayType_corridor,...,TimeToSubway_5min~10min,TimeToSubway_no_bus_stop_nearby,SubwayStation_Bangoge,SubwayStation_Banwoldang,SubwayStation_Chil-sung-market,SubwayStation_Daegu,SubwayStation_Kyungbuk_uni_hospital,SubwayStation_Myung-duk,SubwayStation_Sin-nam,SubwayStation_no_subway_nearby
0,141592,2006,814,3,3.0,0.0,5,6.0,9.0,0,...,0,0,0,0,0,0,1,0,0,0
1,51327,1985,587,8,2.0,2.0,3,12.0,4.0,1,...,1,0,0,0,0,1,0,0,0,0
2,48672,1985,587,6,2.0,2.0,3,12.0,4.0,1,...,1,0,0,0,0,1,0,0,0,0
3,380530,2006,2056,8,5.0,11.0,5,3.0,7.0,0,...,0,0,0,0,0,0,0,0,1,0
4,78318,1992,644,2,4.0,8.0,3,9.0,14.0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4119,570796,2007,1928,24,14.0,16.0,10,9.0,10.0,0,...,0,0,0,0,0,0,1,0,0,0
4120,307079,2015,644,22,5.0,10.0,7,7.0,11.0,0,...,1,0,0,0,0,1,0,0,0,0
4121,357522,2007,868,20,14.0,16.0,10,9.0,10.0,0,...,0,0,0,0,0,0,1,0,0,0
4122,312389,1978,1327,1,1.0,4.0,3,7.0,11.0,1,...,0,0,0,0,0,0,1,0,0,0


In [7]:
def price_to_class(price: int) -> int:
    return int(price > 350_000) + int(price > 100_000)

features = df.drop(columns=['SalePrice'])
labels = pd.get_dummies(df['SalePrice'].rename('class').apply(price_to_class))
labels

,0,1,2
0,False,True,False
1,True,False,False
2,True,False,False
3,False,False,True
4,True,False,False
...,...,...,...
4119,False,False,True
4120,False,True,False
4121,False,False,True
4122,False,True,False


In [8]:
dataset = ApartmentDataset(features=features.values, labels=labels.values)
data_module = ApartmentDataModule(dataset=dataset)

In [9]:
model = ApartmentClassifier(input_size=len(features.columns), output_size=3)

In [10]:
trainer = pl.Trainer(max_epochs=200)
trainer.fit(model, datamodule=data_module)

# `tensorboard --logdir=lightning_logs` to launch tensorboard

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type             | Params
------------------------------------------
0 | l1   | Linear           | 9.6 K 
1 | l2   | Linear           | 8.3 K 
2 | l3   | Linear           | 2.1 K 
3 | l4   | Linear           | 99    
4 | loss | CrossEntropyLoss | 0     
------------------------------------------
20.0 K    Trainable params
0         Non-trainable params
20.0 K    Total params
0.080     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/tomek/miniconda3/envs/ssne_p2/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (7) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.
